# COBAR 2023 Locomotion Code Submission

Please use this template to submit your code for your COBAR miniproject.

**PLEASE READ THE INSTRUCTIONS BELOW CAREFULLY.**

**Google Colab:** It is your responsibility to make sure that your code can be executed without error on Google Colab. You may use your own machine for development, but in the end we will use Colab to determine whether your code runs. You need to select a GPU instance on Google Colab for this notebook. Go to "Runtime" > "Change runtime type", select "GPU" under "Hardware accelerator" and click "Save."

**Where to put your code:** Please only fill your code in places indicated by the following. These cells are provided flexibly enough, and you should be able to easily add your custom import/loading/preprocessing/stepping code. You may also use common `*.py` module files and import functions/classes from these modules.
```Python
# ========== YOUR CODE BELOW ==========
... write your code here
# ^^^^^^^^^^ END OF YOUR CODE ^^^^^^^^^
```

**Terrain types:** Once you have completed this notebook for flat terrain, duplicate 2 more copies of this notebook, and change the terrain type to "gapped" and "blocks" respectively. Make sure you can run all three notebooks on Colab.


**What to submit:** Please submit the following files via Moodle:
1. The three *executed* notebooks (ie. the outputs of the code blocks are printed). Name these files "submission_flat.ipynb", "submission_gapped.ipynb", and "submission_blocks.ipynb"
2. The three output videos: "output_flat.mp4", "output_gapped.mp4", and "output_blocks.mp4"
3. Any data file that are required for the notebooks to run (eg. parameter files, additional configurations, ...). In other words, include any file that you are loading to the notebooks. We will download these files to the same directory/folder as the notebook files before running the notebooks.
4. If you have any `*.py` module file from which you imported variables, functions, or classes, included these files too.

---

In [ ]:
# #@title Install `flygym` on Colab

# # This block is modified from dm_control's tutorial notebook
# # https://github.com/deepmind/dm_control/blob/main/tutorial.ipynb

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    import subprocess
    if subprocess.run('nvidia-smi').returncode:
        raise RuntimeError(
            'Cannot communicate with GPU. '
            'Make sure you are using a GPU Colab runtime. '
            'Go to the Runtime menu and select Choose runtime type.')

    print('Installing flygym')
    !pip install -q --progress-bar=off 'flygym[mujoco] @ git+https://github.com/NeLy-EPFL/flygym.git'

    # Configure dm_control to use the EGL rendering backend (requires GPU)
    %env MUJOCO_GL=egl

    print('Checking that the dm_control installation succeeded...')
    try:
        from dm_control import suite
        env = suite.load('cartpole', 'swingup')
        pixels = env.physics.render()
    except Exception as e:
        raise e from RuntimeError(
            'Something went wrong during dm_control installation. Check the shell '
            'output above for more information.\n'
            'If using a hosted Colab runtime, make sure you enable GPU acceleration '
            'by going to the Runtime menu and selecting "Choose runtime type".')
    else:
        del pixels, suite

    print('Checking that the flygym installation succeeded...')
    try:
        import flygym
        from flygym import envs
    except Exception as e:
        raise e from RuntimeError(
            'Something went wrong during flygym installation. Check the shell '
            'output above for more information.\n')
    else:
        del envs, flygym
else:
    print('Skipping - not on Colab')

You may install any additional packages here with pip. NumPy should already be installed; it's merely a demonstration of the syntax.

In [ ]:
# ========== YOUR CODE BELOW ==========
!pip install numpy
# ^^^^^^^^^^ END OF YOUR CODE ^^^^^^^^^

The following are some of the modules imported in the demo notebooks:

In [ ]:
import numpy as np
import pkg_resources
import pickle
import matplotlib.pyplot as plt
from pathlib import Path
from flygym.envs.nmf_mujoco import NeuroMechFlyMuJoCo
from tqdm import trange
from flygym.util.config import all_leg_dofs

You may import any additional modules here:

In [ ]:
# ========== YOUR CODE BELOW ==========

# ^^^^^^^^^^ END OF YOUR CODE ^^^^^^^^^

We select the terrain type here. **YOUR CODE MUST STILL WORK IF THIS IS CHANGED TO gapped OR blocks!**

In [ ]:
terrain_type = 'flat'  # or 'gapped' or 'blocks'

We define some other physics parameters. Thess must not be changed.

In [ ]:
run_time = 1
out_dir = Path(f'contest_submission_{terrain_type}')

friction = 1.0

physics_config = {
    'joint_stiffness': 2500,
    'friction': (friction, 0.005, 0.0001),
    'gravity': (0, 0, -9.81e5)}
terrain_config = {'fly_pos': (0, 0, 300),
                  'friction': (friction, 0.005, 0.0001)}

You may load any preprogrammed data here and manipulate them as needed. This can include reference step kinematics (as used in the decentralized control and centralized control notebooks), or parameter files for your artificial neural network:

In [ ]:
# ========== YOUR CODE BELOW ==========
    
# ^^^^^^^^^^ END OF YOUR CODE ^^^^^^^^^

Next, initiate your NeuroMechFly simulation environment.

In [ ]:
# ========== YOUR CODE BELOW ==========
nmf = ...
# ^^^^^^^^^^ END OF YOUR CODE ^^^^^^^^^

You can add arbitrary code here after initializing the `nmf` object and before running the simulation loop:

In [ ]:
# ========== YOUR CODE BELOW ==========

# ^^^^^^^^^^ END OF YOUR CODE ^^^^^^^^^

Now, run the simulation:

In [ ]:
# Final preparation here
# ========== YOUR CODE BELOW ==========

# ^^^^^^^^^^ END OF YOUR CODE ^^^^^^^^^

nmf.reset()
x_pos_hist = []

# Main simulation loop
for i in trange(num_steps):
    # You may insert any code before stepping the physics simulation...
    # ========== YOUR CODE BELOW ==========
    
    # ^^^^^^^^^^ END OF YOUR CODE ^^^^^^^^^
    
    obs, info = nmf.step(action)
    nmf.render()
    
    # ... and after stepping the physics simulation
    # ========== YOUR CODE BELOW ==========
    
    # ^^^^^^^^^^ END OF YOUR CODE ^^^^^^^^^
    
    # Report the final x position of the base of the fly here. If you wrote
    # a wrapper around `NeuroMechFlyMuJoCo` (for your own RL pipeline, for
    # example), you may modify the code below to faithfully report the final
    # x position of the base of the fly depending on your implementation.
    # Obviously, don't hard code a very large number :)
    # ========== YOUR CODE BELOW ==========
    current_x_pos = obs['fly'][0, 0]
    # ^^^^^^^^^^ END OF YOUR CODE ^^^^^^^^^
    x_pos_hist.append(current_x_pos)

# Save video here. SUBMIT THIS VIDEO ALONG WITH THE CODE!
nmf.save_video(out_dir / f"output_{terrain_type}.mp4")

nmf.close()


The video should have been saved to `out_dir / f"output_{terrain_type}.mp4"`. Check that it makes sense.

Finally, let's visualize how the fly's x position changes over time:

In [ ]:
plt.plot(np.arange(num_steps) * nmf.timestep, np.array(x_pos_hist) / 1000)
plt.xlabel('Time (s)')
plt.ylabel('x position (mm)')
print(f'Final position: {x_pos_hist[-1] / 1000:.4f} mm')